In [1]:
from pathlib import Path
import pickle
import ipdb
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import jax.numpy as jnp
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def edge_exists(graph, node1, node2):
    senders = graph.senders
    receivers = graph.receivers
    exists = jnp.any((senders == node1) & (receivers == node2))
    return exists

In [3]:
def is_independent_set(jgraph, independent_set):
    # is_MIS = True
    # for send, receive in zip(jgraph.senders, jgraph.receivers):
    #     if independent_set[send] == 1 and independent_set[receive] == 1:
    #         is_MIS = False
    # # if is_MIS:
    # #     f0.write(f"The size is {sum(independent_set)}")
    # return is_MIS
    senders = jgraph.senders
    receivers = jgraph.receivers
    n = len(independent_set)
    for i in range(n):
        for j in range(i, n):
            if jnp.any((senders == i) & (receivers == j)):
                return False
    
    return True

In [4]:
def count_voilate(jgraph, independent_set):
    is_MIS = True
    edges = []
    voilate_cnt = 0
    for send, receive in zip(jgraph.senders, jgraph.receivers):
        if (send, receive) in edges or (receive, send) in edges:
            continue
        edges.append((send, receive))
        if independent_set[send] == 1 and independent_set[receive] == 1:
            voilate_cnt += 1
    return voilate_cnt

In [5]:
# def calc_IS_size():
#     path = ""
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/z5xmvb6v/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization_KS_3_1000")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/spcrakpu/visualization_KS_3_1000")
# path = Path("/home/haojun/DIffUCO/Checkpoints/71qtteyy/visualization_KS_3_1000_train")
# path = Path("/home/haojun/DIffUCO/Checkpoints/ojqw39ks/visualization_KS_3_1000_train/4x5")
# path = Path("/home/haojun/DIffUCO/Checkpoints/f3ad84ee/visualization_KS_one_3_1000/6x5")
# path = Path("/home/haojun/DIffUCO/Checkpoints/2cdb71a6/visualization_KS_one_3_1000/6x5")
# path = Path("/home/haojun/DIffUCO/Checkpoints/2a4a042f/visualization_KS_one_3_1000/6x3")
# path = Path("/home/haojun/DIffUCO/Checkpoints/2efec818/visualization_KS_one_3_1000/4x3")
# path = Path("/home/haojun/DIffUCO/Checkpoints/spcrakpu/visualization_KS_one_3_1000/3x8")
# path = Path("/home/haojun/DIffUCO/Checkpoints/spcrakpu/visualization_KS_one_3_1000/3x20")
# path = Path("/home/haojun/DIffUCO/Checkpoints/f53af0da/visualization_KS_5_5000/4x5")
# path = Path("/home/haojun/DIffUCO/Checkpoints/0ba01942/visualization_KS_5_5000/4x4")
# path = Path("/home/haojun/DIffUCO/Checkpoints/026ae66b/visualization_KS_5_5000/8x5")
# path = Path("/home/haojun/DIffUCO/Checkpoints/2992ad1d/visualization_KS_5_5000/4x3")
path = Path("/home/haojun/DIffUCO/Checkpoints/31e75545/visualization_KS_5_5000/4x3")
cnt = []
tot_nodes = []
iter = 0
log_path = path / "log.txt"
with open(log_path, 'w+') as f0:
    for graph_dir in tqdm(path.iterdir()):
        if not graph_dir.is_dir():
            continue
        f0.write(f"graph_dir={graph_dir}\n")

        jgraph_path = graph_dir / "jgraph.pkl"
        solution_path = graph_dir / "solution.pkl"

        with open(jgraph_path, 'rb') as f:
            jgraph = pickle.load(f)

        with open(solution_path, 'rb') as f:
            solution = pickle.load(f)

        # G = nx.DiGraph()
        # for i, nodes in enumerate(jgraph.nodes):
        #     G.add_node(i)
            
        # for send, receive in zip(jgraph.senders, jgraph.receivers):
        #     G.add_edge(send, receive)
        
        # f0.write(f"solution 's size={len(solution.flatten())}")
        f0.write(f"iter: {iter}\n")
        iter += 1

        independent_set = solution.flatten()[:len(jgraph.nodes)]

        cnt.append(sum(independent_set))
        tot_nodes.append(len(independent_set))
        # f0.write(f"the number of violate edges = {count_voilate(jgraph, independent_set)}")
        f0.write(f"size of independent set = {sum(independent_set)}\n")
        f0.write(f"size of nodes = {len(independent_set)}\n")
        voilate_edges = []
        senders = jgraph.senders
        receivers = jgraph.receivers
        independent_set_true_idx = jnp.nonzero(independent_set)[0]
        n = len(independent_set_true_idx)
        voilate_cnt = 0
        f0.write(f"independent_set_true_idx={independent_set_true_idx}\n")
        voilate_idx = []
        for i in range(n):
            for j in range(i + 1, n):
                a = independent_set_true_idx[i]
                b = independent_set_true_idx[j]
                if jnp.any((senders == a) & (receivers == b)):
                    voilate_cnt += 1
                    voilate_edges.append((a.item(), b.item()))
                    if not a.item() in voilate_idx: voilate_idx.append(a.item())
                    if not b.item() in voilate_idx: voilate_idx.append(b.item())
                elif jnp.any((senders == b) & (receivers == a)):
                    voilate_cnt += 1
                    voilate_edges.append((b.item(), a.item()))
                    if not a.item() in voilate_idx: voilate_idx.append(a.item())
                    if not b.item() in voilate_idx: voilate_idx.append(b.item())
        f0.write(f"voilate_idx={voilate_idx}\n")
        f0.write(f"voilate_cnt={voilate_cnt}\n")
        f0.write(f"After eliminating, solution_size={sum(independent_set) - len(voilate_idx)}\n")
        voilate_path = graph_dir / "voilate.pkl"
        with open(str(voilate_path), 'wb') as f2:
            pickle.dump(voilate_edges, f2)
        f0.write(f"-" * 8 + '\n')
    

0it [00:00, ?it/s]

2024-07-29 12:37:44.214835: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
13it [00:19,  1.48s/it]
